**Send tweets from the Twitter API**

Import the necessary packages

In [ ]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

Insert your credentials

In [ ]:
consumer_key='AL7VpdwuBLHuumpxLkV0dzEYY'
consumer_secret='lZwNGBLDgR0qB5Y8HICS6qv5DaqSosMSXTOpahSXTxsJYMVf7A'
access_token ='1226360261428121600-kLVvbZaZFxiQ4w1zvne4vj7tucPVm4'
access_secret='p7EZsyK7WvZ60kkcc3kH3xyG3kx6FI6VbHbQFtoH30rQO'